In [95]:
from absbox.local.generic import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-02-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":30
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None)

In [96]:
from absbox import API
localAPI = API("https://absbox.org/api/dev")
#localAPI = API("http://localhost:8081")

In [97]:
# pool performance
pool_assumps = {
     "CPR15":[{"CPR":0.15}]
    ,"CPR20":[{"CPR":0.20}]
    ,"CPR25":[{"CPR":0.25}]
}
# pricing curves and PV date
pricing_assumps = {"PVDate":"2021-08-22"
                  ,"PVCurve":[["2021-01-01",0.025]
                             ,["2024-08-01",0.025]]}

In [98]:
# impor the candy function
from absbox.local.analytics import run_yield_table

#from absbox import API
#localAPI = API("https://absbox.org/api/latest")


# test01 is a deal object
run_yield_table(localAPI, test01, "A1", pool_assumps, pricing_assumps )

,CPR15,CPR20,CPR25
pricing,541.520,498.050,453.080
face,54.152,49.805,45.308
WAL,0.280,0.250,0.220
duration,0.240,0.220,0.190
convexity,0.360,0.310,0.270
accure interest,0.000,0.000,0.000


In [100]:
issuance_plan = [ (1100,0.07),(1500,0.08) ]
total_issuance_bal = 2000

bond_plan = [ {"bonds":(("A1",{"balance":senior_bal
                          ,"rate":senior_r
                          ,"originBalance":senior_bal
                          ,"originRate":0.07
                          ,"startDate":"2020-01-03"
                          ,"rateType":{"Fixed":0.08}
                          ,"bondType":{"Sequential":None}})
                   ,("B",{"balance":(total_issuance_bal - senior_bal)
                          ,"rate":0.0
                          ,"originBalance":(total_issuance_bal - senior_bal)
                          ,"originRate":0.07
                          ,"startDate":"2020-01-03"
                          ,"rateType":{"Fixed":0.00}
                          ,"bondType":{"Equity":None}
                          }))}
     for senior_bal,senior_r in issuance_plan ]

In [101]:
bond_plan_with_name = dict(zip(["SmallSenior","HighSenior"],bond_plan))

from absbox.local.util import mkDealsBy

differentDeals = mkDealsBy(test01,bond_plan_with_name)


In [102]:
r = localAPI.runStructs(differentDeals
                        ,read=True
                        ,assumptions=[{"CDR":[0.01,0.015,0.021,0.021]}
                                    ,{"Recovery":(0.7,1)}]
                        )
r

{'HighSenior': {'bonds': OrderedDict([('A1',
                            balance  interest  principal  rate    cash  \
                date                                                     
                2021-07-26  1228.16     13.47     271.84  0.08  285.31   
                2021-08-20  1155.16      6.72      73.00  0.08   79.72   
                2021-09-20  1085.76      7.84      69.40  0.08   77.24   
                2021-10-20  1015.66      7.13      70.10  0.08   77.23   
                2021-11-20   945.33      6.90      70.33  0.08   77.23   
                2021-12-20   874.30      6.21      71.03  0.08   77.24   
                2022-01-20   803.01      5.94      71.29  0.08   77.23   
                2022-02-20   731.23      5.45      71.78  0.08   77.23   
                2022-03-20   658.47      4.48      72.76  0.08   77.24   
                2022-04-20   585.70      4.47      72.77  0.08   77.24   
                2022-05-20   512.31      3.85      73.39  0.08   77

In [103]:
r = localAPI.run(differentDeals['HighSenior']
                        ,read=True
                        ,assumptions=[{"CDR":[0.01,0.015,0.021,0.021]}
                                    ,{"Recovery":(0.7,1)}]
                        )
r["pool"]['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum
Date,,,,,,,,,
2021-06-15,1910.03,278.40,31.32,0,11.57,5.59,2.38,0.045,None
2021-07-31,1839.96,70.07,7.16,0,0.00,2.49,1.07,0.045,None
2021-08-31,1769.61,70.35,6.89,0,0.00,0.00,0.00,0.045,None
2021-09-30,1699.01,70.60,6.63,0,0.00,0.00,0.00,0.045,None
2021-10-31,1628.15,70.86,6.37,0,0.00,0.00,0.00,0.045,None
2021-11-30,1557.01,71.14,6.10,0,0.00,0.00,0.00,0.045,None
2021-12-31,1485.61,71.40,5.83,0,0.00,0.00,0.00,0.045,None
2022-01-31,1413.95,71.66,5.57,0,0.00,0.00,0.00,0.045,None
2022-02-28,1342.01,71.94,5.30,0,0.00,0.00,0.00,0.045,None


In [19]:
r['SmallSenior']['pricing']

,pricing,face,WAL,duration,convexity,accure interest
A1,568.69,51.699090,0.24,0.21,0.3,0
B,1103.14,122.571111,0.78,0.78,1.6,0


In [9]:
r['HighSenior']['pricing']

,pricing,face,WAL,duration,convexity,accure interest
A1,1110.72,74.048,0.45,0.39,0.67,0
B,675.57,135.114,1.04,1.04,2.27,0


In [10]:
test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None)

In [13]:
localAPI.run(test01,
     assumptions=[],
     read=True)

{'bonds': OrderedDict([('A1',
                           balance  interest  principal  rate    cash  \
               date                                                     
               2021-07-26   566.42      7.86     433.58  0.07  441.44   
               2021-08-20   451.27      2.71     115.15  0.07  117.86   
               2021-09-20   336.09      2.68     115.18  0.07  117.86   
               2021-10-20   220.16      1.93     115.93  0.07  117.86   
               2021-11-20   103.60      1.30     116.56  0.07  117.86   
               2021-12-20     0.00      0.59     103.60  0.07  104.19   
               
                                                                        memo  
               date                                                           
               2021-07-26  [<PayInt:["A1"],Just 0.00>, <PayPrin:["A1"],Ju...  
               2021-08-20  [<PayInt:["A1"],Just 0.00>, <PayPrin:["A1"],Ju...  
               2021-09-20  [<PayInt:["A1"],Just 0.00>,